In [10]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # setting ignore as a parameter and further adding category
from whispr import *
import os


In [11]:
# gene expression. 2.5ul rxns, 75% txtl
rxn_vol = 5
source_plate_type = '384PP_AQ_BP' 

os.getcwd()
folder = os.getcwd() + '/Experiments/230207_mod1/'
sp_plasmids_file = folder + 'plasmids_sp.xlsx'
sp_plasmids = pd.read_excel(sp_plasmids_file, index_col = 0, engine='openpyxl')
sp_plasmids = sp_plasmids[~sp_plasmids['Well'].isna()]

layout_genex_file = folder + 'genex-pl-96.csv'
layout_genex = pd.read_csv(layout_genex_file, index_col = 0, dtype = str)

mt_genex_file = folder + 'genex_mt.csv'
mt_genex = pd.read_csv(mt_genex_file, index_col = 0, dtype = str).fillna(0)

checkInputs(sp_plasmids, mt_genex,source_plate_type)

vol_table_df = generateVolumeTable(mt_genex, sp_plasmids, rxn_vol = 0.25*rxn_vol, total_vol = rxn_vol, fill_with='water', multiRpW = False)

#specify rxn_vol (default = 2.5) and total_vol (default = 10) if you'd like to change the volume of each individual replicate or the total reaction volume
protocol_genex_df = writeProtocol(source_plate_type, vol_table_df, layout_genex,sp_plasmids, update_source_vol=folder+'230207_plasmids_sp_updated.xlsx')
protocol_genex_df = protocol_genex_df[0]


Volumes used from each well for this protocol:
{'1_A1': 10.025, '1_P3': 0.225, '1_P9': 0.8, '1_P15': 0.425, '1_P17': 0.1, '1_P18': 0.075, '1_O1': 0.025, '1_O2': 0.025, '1_O3': 0.05, '1_O4': 0.075, '1_O5': 0.05, '1_O6': 0.025, '1_O7': 0.175, '1_O8': 0.025, '1_O9': 0.075, '1_O11': 0.1, '1_O12': 0.225}


In [12]:
vol_table_df

,Label,water,biotin,Pyc,Fdh,P70_GFP,Mdh,Mdh_10x,ME2,TdcB_old,...,mtdA,fdhAT,gcvP,gcvT,gcvH,lpd,lplA,fdhCB,glyA,TdcB
0,TdcB_plus,0.925,0,0,0,NaN,0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,NaN,0,0.225
1,neg,1.15,0,0,0,NaN,0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,NaN,0,0
2,glyA_plus,1.05,0,0,0,NaN,0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,NaN,0.1,0
3,ppk12_plus,0.725,0,0,0,NaN,0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,NaN,0,0
4,ppkMt_plus,1.05,0,0,0,NaN,0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,NaN,0,0
5,ppkPa_plus,1.075,0,0,0,NaN,0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,NaN,0,0
6,mod1_pos,1.025,0,0,0,NaN,0,NaN,NaN,NaN,...,0.05,0.025,0,0,0,0,0,NaN,0,0
7,mod2_pos,0.7,0,0,0.1,NaN,0,NaN,NaN,NaN,...,0,0,0.05,0.025,0.175,0.025,0.075,NaN,0,0
8,fdhAT,1.2,0,0,0,NaN,0,NaN,NaN,NaN,...,0,0.05,0,0,0,0,0,NaN,0,0
9,fdhSN,1.125,0,0,0.125,NaN,0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,NaN,0,0


In [13]:
# everything gets txtl

# get destination wells
for layout_id, layout in enumerate([layout_genex]):    
    # reads plate layout and assigns wells to each reaction (rxn_loc; dict)
    labels = pd.unique(np.concatenate(layout.values))
    rxn_loc = {}
    for l in labels:
        if type(l) is str:
            index = layout[layout.isin([l])].stack().index
            rxn_loc[l] = []
            for i in index:
                rxn_loc[l].append(str(i[0]) + str(i[1]))


txtl_well = 'D24'
for label, dwell in rxn_loc.items():
    for d in dwell:
        print(label)
        txtl_vol = vol_table_df[vol_table_df['Label'] == label].iloc[0,1:].sum()/0.25*0.75*1000 # add txtl to make 75%
        row = {'Source Plate Name':'Source[1]', 'Source Plate Type': source_plate_type, 'Source Well': txtl_well,
                                    'Destination Plate Name':'Destination[1]', 'Destination Well': d, 'Transfer Volume': txtl_vol}
        protocol_genex_df = protocol_genex_df.append(row, ignore_index = True)

protocol_genex_df.to_csv(folder+'protocol_genex.csv',index = False)
print('TXTL needed: '+str(protocol_genex_df[protocol_genex_df['Source Well'] == txtl_well]['Transfer Volume'].sum()/1000))
# carry out gene expression reaction

neg
ppkPa_plus
TdcB_plus
ppk12_plus
glyA_plus
ppkMt_plus
mod1_pos
fdhAT
mod2_pos
fdhSN
TXTL needed: 37.5


In [6]:
# dilution. everything gets Tris, for 20x -> 57uL

##### doing it by hand if not too many reactions #####

# protocol_dilution_df = pd.DataFrame(columns=protocol_genex_df.columns)
# source_plate_type = '6RES_AQ_B' 
# tris_well = 'A1'
# tris_vol = 57*1000

# for dwell in protocol_genex_df['Destination Well'].unique():
#     row = {'Source Plate Name':'Source[1]', 'Source Plate Type': source_plate_type, 'Source Well': tris_well,
#                                 'Destination Plate Name':'Destination[1]', 'Destination Well': dwell, 'Transfer Volume': tris_vol}
#     protocol_dilution_df = protocol_dilution_df.append(row, ignore_index = True)

# protocol_dilution_df.to_csv('protocols/221024_arpae_dilutions.csv',index = False)

In [7]:
# from source plate with buffer, mix into 96

# from source plate with genex, mix the diluted txtl

In [14]:
# biosynthesis. 25ul rxns, 2.5ul of diluted txtl

# buffer source plate
sp_buffers_file = folder + 'buffers_sp.xlsx'
sp_buffers= pd.read_excel(sp_buffers_file, index_col = 0, engine = 'openpyxl')
sp_buffers = sp_buffers[~sp_buffers['Well'].isna()]

# HEPES reservoir (actually same plate as Tris)
sp_hepes = pd.DataFrame(columns=sp_buffers.columns)
sp_hepes = sp_hepes.append({'Label':'HEPES', 'Well':'A2,A3', 'Concentration':'', 'Volume':'2000,2000'}, ignore_index = True)

# gene expression source plate (diluted w/ Tris)
#sp_genex = sp_from_layout(layout_genex, 60)
layout_genex_384_file = folder + 'genex-pl-384.csv'
layout_genex_384 = pd.read_csv(layout_genex_384_file, index_col = 0, dtype = str)
sp_genex = sp_from_layout(layout_genex_384, 55)


# group source plates
sp_types = ['384PP_AQ_BP','6RES_AQ_BP2','384PP_AQ_BP'] # triple check this
sps = [sp_buffers,sp_hepes,sp_genex]

# get mixing table
mt_biosyn_file = folder + 'buffers-mt.csv'
mt_biosyn = pd.read_csv(mt_biosyn_file, index_col = 0, dtype = str).fillna(0)
mt_biosyn = mt_biosyn[~mt_biosyn.index.isna()]

# check formats
checkInputs(sps,mt_biosyn,sp_types)

# layouts for destination plate(s)
layout_biosyn1_file = folder + 'biosyn_pl.csv' 
layout_biosyn1 = pd.read_csv(layout_biosyn1_file, index_col = 0, dtype = str)
layout_biosyn1 = layout_biosyn1[~layout_biosyn1.index.isna()]

# filename = '221018_arpae_biosyn_neg.csv'
# layout_biosyn2_file = 'plate_layouts/'+filename 
# layout_biosyn2 = pd.read_csv(layout_biosyn2_file, index_col = 0, dtype = str)
# layouts = [layout_biosyn1,layout_biosyn2]

vol_table_df = generateVolumeTable(mt_biosyn, sps, rxn_vol = 22.5, total_vol = 25, fill_with='HEPES')

protocol_biosyn_dfs = writeProtocol(sp_types, vol_table_df, layout_biosyn1,sps, update_source_vol= folder +'combined_sp_updated.xlsx')

#protocol_biosyn_df.to_csv('protocols/221018_arpae_biosyn.csv',index = False)

Volumes used from each well for this protocol:
{'1_B1': 4.0, '1_F1': 0.3, '1_G1': 13.2, '1_H1': 4.5, '1_I1': 0.825, '1_J1': 10.0, '1_M1': 30.0, '1_N1': 3.0, '1_O1': 10.5, '1_P1': 6.0, '1_B3': 2.0, '1_P3': 14.0, '1_O3': 5.5, '1_N3': 7.0, '1_M3': 8.5, '1_D4': 2.25, '1_P4': 1.25, '1_J4': 0.5, '1_M4': 10.0, '1_A4': 9.375, '1_C4': 0.45, '2_A2': 1076.85, '3_A12': 35.0, '3_A13': 27.5, '3_B12': 7.5, '3_C12': 7.5, '3_D12': 7.5, '3_E12': 15.0, '3_F12': 7.5, '3_G12': 7.5, '3_H12': 7.5, '3_I12': 15.0, '3_J12': 15.0}


In [16]:
protocol_biosyn_dfs[0].to_csv(folder+'biosyn_protocol_384.csv',index = False)
protocol_biosyn_dfs[1].to_csv(folder+'biosyn_protocol_6RES.csv',index = False)

In [17]:
sps[2]

,Concentration,Label,Volume,Well
Item,,,,
neg,100.0,neg,55,J11
ppkPa_plus,100.0,ppkPa_plus,55,J12
TdcB_plus,100.0,TdcB_plus,55,K11
ppk12_plus,100.0,ppk12_plus,55,K12
glyA_plus,100.0,glyA_plus,55,L11
ppkMt_plus,100.0,ppkMt_plus,55,L12
mod1_pos,100.0,mod1_pos,55,M11
fdhAT,100.0,fdhAT,55,M12
mod2_pos,100.0,mod2_pos,55,N11
